In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [ ]:
# len特征计算
# my_feat_calc_func 函数是并行计算特征的函数
# 只需要将包含queryid、query、title列的dataframe传入函数my_feat_calc_func()，即可计算出该类特征

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 2, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def apply_fun(df):
    basic_feat=pd.DataFrame()
    
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    basic_feat['querykw_num']=df['query'].apply(lambda x:len(x.split(sep=" ")))
    basic_feat['titlekw_num']=df['title'].apply(lambda x:len(x.split(sep=" ")))
    
    return basic_feat

def my_feat_calc_func(RawData):
    '''  关键词长度相关特征  a'''
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# query_pos_{1-10}_feat
# my_feat_calc_func 函数是并行计算特征的函数

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def keyWordMatchfunc(doc1,doc2,pos_i):
    doc1=doc1.split(sep=" ")
    doc2=doc2.split(sep=" ")
    
    if pos_i<len(doc1):
        title_len=min(len(doc2),25) # title只匹配前25个关键词
        for pos_j in range(title_len):
            if doc1[pos_i]==doc2[pos_j]:
                q_pos=pos_j+1       # 如果匹配上了 记录匹配的位置
                break
            elif pos_j==title_len-1:
                q_pos=0             # 如果没有匹配上 赋值为0
    else:
        q_pos=-1                    # 如果后续长度不存在 赋值为-1 
    
    return q_pos

def apply_fun(df):
    basic_feat=pd.DataFrame()
    
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    query_pos_feat=pd.DataFrame()
    
    for pos_i in range(10):
        query_pos_feat['query_pos_'+str(pos_i+1)] = df.apply(
                lambda row:keyWordMatchfunc(row["query"],row.title,pos_i),axis=1).astype(np.int8)
    
    return query_pos_feat

def my_feat_calc_func(RawData):
    '''  关键词长度相关特征  a'''
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# title_pos_{1-25}_feat
# my_feat_calc_func 函数是并行计算特征的函数

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def keyWordMatchfunc(doc2,doc1,pos_i):
    doc1=doc1.split(sep=" ")
    doc2=doc2.split(sep=" ")
    
    if pos_i<len(doc1):
        
        title_len=len(doc2)         # title只匹配前25个关键词
        for pos_j in range(title_len):
            if doc1[pos_i]==doc2[pos_j]:
                t_pos=pos_j+1       # 如果匹配上了 记录匹配的位置
                break
            elif pos_j==title_len-1:
                t_pos=0             # 如果没有匹配上 赋值为0
    else:
        t_pos=-1                    # 如果后续长度不存在 赋值为-1 
    
    return t_pos

def apply_fun(df):
    basic_feat=pd.DataFrame()
    
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    title_pos_feat=pd.DataFrame()
    
    for pos_i in range(15):
        title_pos_feat['title_pos_'+str(pos_i+1)] = df.apply(
                lambda row:keyWordMatchfunc(row["query"],row.title,pos_i),axis=1).astype(np.int)
    
    return title_pos_feat

def my_feat_calc_func(RawData):
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# sim_feat 细粒度的特征
# my_feat_calc_func 函数是并行计算特征的函数

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

import distance
def apply_fun(df):
    basic_feat=pd.DataFrame()
    
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    sim_feat=pd.DataFrame()
    # qt=[[10,10],[10,15],[15,15],[15,25]]
    # qt=[[3,5],[3,10],[4,5],[4,10],[5,5],[5,10],[10,5],[10,10]]
    qt=[[3,3],[3,5],[5,5],[5,10],[10,10],[10,15],[15,15],[15,25]]
    
    sim_func_dict={"jaccard":distance.jaccard,
                    # "sorensen":distance.sorensen,
                    "levenshtein":distance.levenshtein,
                    # "ratio":Levenshtein.ratio
    }
    
    for sim_func in tqdm(sim_func_dict):
        sim_feat[sim_func] = df.apply(lambda row:sim_func_dict[sim_func](row["query"].split(sep=" "),
                                                row["title"].split(sep=" ")),
                                                axis=1)
        for qt_len in tqdm(qt):
            if qt_len[0]==3 and sim_func=="levenshtein":
                pass
            else:
                sim_feat[sim_func+'_q'+str(qt_len[0])+'_t'+str(qt_len[1])] = df.apply(
                    lambda row:sim_func_dict[sim_func]( row["query"].split(sep=" ")[:qt_len[0]],
                                                        row["title"].split(sep=" ")[:qt_len[1]]),
                                                        axis=1)
    
    return sim_feat

def my_feat_calc_func(RawData):
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# BM25特征
# my_feat_calc_func 函数是并行计算特征的函数
# 只需要将包含queryid、query、title列的dataframe传入函数my_feat_calc_func()，即可计算出该类特征
# 将前一亿、后一亿、test1、test2传入dataframe

In [ ]:
import pandas as pd
from gensim.summarization.bm25 import BM25

from tqdm import tqdm

# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

import time

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def apply_fun(df):
    query_id_group = df.groupby(['query_id'])
    bm_list = []
    for name, group in query_id_group:
        corpus = group['title'].values.tolist()
        corpus = [sentence.strip().split() for sentence in corpus]
        query = group['query'].values[0].strip().split()
        bm = BM25(corpus)
        bmscore = bm.get_scores(query)
        bm_list.extend(bmscore)
    
    return pd.DataFrame(bm_list,columns=["BM25"])

def my_feat_calc_func(RawData):
    '''  关键词长度相关特征  a'''
    multiprocessing_nums=13
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# 计算query_match_feat 特征
# my_feat_calc_func 函数是并行计算特征的函数
# 只需要将包含queryid、query、title列的dataframe传入函数my_feat_calc_func()，即可计算出该类特征
# 将前一亿、后一亿、test1、test2传入dataframe

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

import time

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def wordblockMatchdel(doc1, doc2):
    doc1 = doc1.split(sep=" ")
    doc2 = doc2.split(sep=" ")
    q1_start = len(doc1)
    q1_end = 0
    count = 0
    blockcount = 0
    i = 0
    j = 0
    maxMatchBlockLen = 0

    while i < len(doc1):
        for j in range(len(doc2)):
            if doc1[i] == doc2[j]:

                q1_start = min(q1_start, j)
                count = count+1
                blockcount = blockcount+1
                maxMatchLen_tmp = 1
                while i+1 < len(doc1) and j + \
                        1 < len(doc2) and doc1[i+1] == doc2[j+1]:
                    maxMatchLen_tmp = maxMatchLen_tmp+1
                    count = count+1
                    i = i+1
                    j = j+1
                maxMatchBlockLen = max(maxMatchBlockLen, maxMatchLen_tmp)
                q1_end = max(q1_end, j)
                break
        i = i+1
    if q1_end-q1_start < 0:
        q1_start = 0
        q1_end = 0
    q1_proximity = q1_end-q1_start
    return [count, blockcount, q1_proximity,
            maxMatchBlockLen, q1_start, q1_end]

def apply_fun(df):
    match_feat_tmp=pd.DataFrame()
    
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    match_feat_tmp["match_tmp"] = df.apply(
            lambda row: wordblockMatchdel(
                row["query"], row.title), axis=1)
    
    match_feat=pd.DataFrame()
    match_feat["count_match"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][0], axis=1)
    
    match_feat["blockcount_match"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][1], axis=1)
    
    match_feat["proximity"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][2], axis=1)
    
    match_feat["maxMatchBlockLen"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][3], axis=1)
    
    match_feat["q1_match_start"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][4], axis=1)
    
    match_feat["q1_match_end"] = match_feat_tmp.apply(
            lambda row: row["match_tmp"][5], axis=1)
    
    return match_feat

def my_feat_calc_func(RawData):
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# fuzz特征
# my_feat_calc_func 函数是并行计算特征的函数

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

from fuzzywuzzy import fuzz
def WRatio(row):
    return fuzz.WRatio(row['query'], row['title'])

def QRatio(row):
    return fuzz.QRatio(row['query'], row['title'])

def partial_token_set_ratio(row):
    return fuzz.partial_token_set_ratio(row['query'], row['title'])

def partial_token_sort_ratio(row):
    return fuzz.partial_token_sort_ratio(row['query'], row['title'])

def partial_ratio(row):
    return fuzz.partial_ratio(row['query'], row['title'])

def token_set_ratio(row):
    return fuzz.token_set_ratio(row['query'], row['title'])

def token_sort_ratio(row):
    return fuzz.token_sort_ratio(row['query'], row['title'])

def apply_fun(df):
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    extracted_feature = pd.DataFrame()
    extracted_feature['token_sort_ratio'] = df.apply(token_sort_ratio, axis=1)
    extracted_feature['token_set_ratio'] = df.apply(token_set_ratio, axis=1)
    extracted_feature['partial_ratio'] = df.apply(partial_ratio, axis=1)
    extracted_feature['partial_token_sort_ratio'] = df.apply(partial_token_sort_ratio, axis=1)
    extracted_feature['partial_token_set_ratio'] = df.apply(partial_token_set_ratio, axis=1)
    extracted_feature['QRatio'] = df.apply(QRatio, axis=1)
    extracted_feature['WRatio'] = df.apply(WRatio, axis=1)
    
    return extracted_feature

def my_feat_calc_func(RawData):
    '''  关键词长度相关特征  a'''
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# textpair 特征
# my_feat_calc_func 函数是并行计算特征的函数
# 只需要将包含queryid、query、title列的dataframe传入函数my_feat_calc_func()，即可计算出该类特征
# 将前一亿、后一亿、test1、test2传入dataframe

In [ ]:
# -*- coding: utf-8 -*- 
'''
Created on Jul 4, 2019

@author: Greatpan
'''

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm
tqdm.pandas(desc='Progress')

from multiprocessing import Pool

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

from fuzzywuzzy import fuzz
def token_set_diff(row):
    query = row['query']
    title = row['title']
    q_set = set(query.split())
    t_set = set(title.split())
    return abs(len(t_set) - len(q_set))

# 词的不同个数之差（有正负）
def wc_diff_unique(row):
    query = row['query']
    title = row['title']
    q_set = set(query.split())
    t_set = set(title.split())
    return len(set(t_set)) - len(set(q_set))

# 词的不同个数比值
def wc_ratio_unique(row):
    query = row['query']
    title = row['title']
    q_set = set(query.split())
    t_set = set(title.split())
    l1 = len(q_set) * 1.0
    l2 = len(t_set) * 1.0
    return l2 / l1

# 总的不同的词
def total_unique_words(row):
    query = row['query']
    query = query.split()
    title = row['title']
    t_set = set(title.split())
    return len(t_set.union(query))

def same_start(row):
    query = row['query']
    title = row['title']
    l = min(len(query), len(title))
    result = 0
    for i in range(l):
        if query[i] == title[i]:
            result += len(title[i])
    return result

def apply_fun(df):
    df["query"]=df["query"].apply(str.strip)
    df["title"]=df["title"].apply(str.strip)
    
    extracted_feature = pd.DataFrame()
    
    extracted_feature['total_unique_words'] = df.apply(total_unique_words, axis=1)
    extracted_feature['wc_ratio_unique'] = df.apply(wc_ratio_unique, axis=1)
    extracted_feature['wc_diff_unique'] = df.apply(wc_diff_unique, axis=1)
    extracted_feature['token_set_diff'] = df.apply(token_set_diff, axis=1)
    extracted_feature['same_start'] = df.apply(same_start, axis=1)
    
    return extracted_feature

def my_feat_calc_func(RawData):
    '''  关键词长度相关特征  a'''
    multiprocessing_nums=16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()
    
    result_parallel = pd.concat(result_parts)
    
    return result_parallel

In [ ]:
# sen句向量相似度/距离 特征
# my_feat_calc_func 函数是并行计算特征的函数
# 只需要将包含queryid、query、title列的dataframe传入函数my_feat_calc_func()，即可计算出该类特征
# 将前一亿、后一亿、test1、test2传入dataframe

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import numpy as np

from tqdm import tqdm

tqdm.pandas(desc='Progress')

from multiprocessing import Pool


def reduce_mem_usage(D, verbose=True):
    start_mem = D.memory_usage().sum() / 1024 ** 2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return D


from gensim.corpora import Dictionary
from gensim.models import TfidfModel, KeyedVectors
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

tqdm.pandas()

dictionary = Dictionary.load('/home/kesci/work/sunrui/tfidf_final/dictionary_final.dic')
print("dic load finish!")
id2token = {id: token for token, id in dictionary.token2id.items()}
print("id2token finish!")
tf_idf_model = TfidfModel.load('/home/kesci/work/sunrui/tfidf_final/tf_idf_final.model')
print("tf_idf_model load finish")
w2v_model = KeyedVectors.load_word2vec_format('/home/kesci/work/word2vec/word2vec_100.bin', binary=True)
print("w2v_model load finish")

fasttext_model = KeyedVectors.load_word2vec_format('./fasttext/fasttext_100.bin', binary=True)
print("fasttext_model load finish")


def get_weight_sentence_vec(tf_idf_vec):
    vec_list = []
    for token_id, tf_idf in tf_idf_vec:
        token = id2token[token_id]
        w2v = w2v_model[token]
        fasttext_vec = fasttext_model[token]
        vec = np.concatenate([w2v, fasttext_vec])
        weighted_vec = vec * tf_idf
        vec_list.append(weighted_vec)
    return np.sum(vec_list, axis=0)


def get_tfidf_vec(text):
    bow = dictionary.doc2bow(text)
    tf_idf = tf_idf_model[bow]
    return tf_idf

from scipy.stats import skew, kurtosis

def compute_sen_vec_dis(query, title):
    tf_idf_q = get_tfidf_vec(query)
    tf_idf_t = get_tfidf_vec(title)
    weighted_q_vec = get_weight_sentence_vec(tf_idf_q)
    weighted_t_vec = get_weight_sentence_vec(tf_idf_t)

    sent_cosine = cosine(weighted_q_vec, weighted_t_vec)
    sent_cityblock = cityblock(weighted_q_vec, weighted_t_vec)
    sent_jaccard = jaccard(weighted_q_vec, weighted_t_vec)
    sent_canberra = canberra(weighted_q_vec, weighted_t_vec)
    sent_euclidean = euclidean(weighted_q_vec, weighted_t_vec)
    sent_minkowski = minkowski(weighted_q_vec, weighted_t_vec)
    sent_braycurtis = braycurtis(weighted_q_vec, weighted_t_vec)

    skew_q = skew(weighted_q_vec)
    skew_t = skew(weighted_t_vec)
    kurtosis_q = kurtosis(weighted_q_vec)
    kurtosis_t = kurtosis(weighted_t_vec)

    return [sent_cosine,sent_cityblock,sent_jaccard,
            sent_canberra,sent_euclidean,sent_minkowski,sent_braycurtis
            skew_q,skew_t,kurtosis_q,kurtosis_t]


def apply_fun(df):
    df["query"] = df["query"].apply(str.strip)
    df["title"] = df["title"].apply(str.strip)

    sen_dis_feat_tmp = pd.DataFrame()
    sen_dis_feat_tmp["sen_dis_tmp"] = df.apply(
        lambda row: compute_sen_vec_dis(
            row['query'].split(),
            row['title'].split()), axis=1)

    sen_dis_feat = pd.DataFrame()
    
    sen_dis_feat_tmp["sen_dis_tmp"] = df.apply(
            lambda row: compute_sen_vec_dis(
                row['query'].split(), 
                row['title'].split()), axis=1)
    
    sen_dis_feat=pd.DataFrame()
    sen_dis_feat["sent_cosine"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][0], axis=1)
    sen_dis_feat["sent_cityblock"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][1], axis=1)
    sen_dis_feat["sent_jaccard"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][2], axis=1)
    sen_dis_feat["sent_canberra"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][3], axis=1)
    sen_dis_feat["sent_euclidean"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][4], axis=1)
    sen_dis_feat["sent_minkowski"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][5], axis=1)
    sen_dis_feat["sent_braycurtis"] = sen_dis_feat_tmp.apply(
            lambda row: row["sen_dis_tmp"][6], axis=1)
    sen_dis_feat["skew_q"] = sen_dis_feat_tmp.apply(
        lambda row: row["sen_dis_tmp"][7], axis=1)
    sen_dis_feat["skew_t"] = sen_dis_feat_tmp.apply(
        lambda row: row["sen_dis_tmp"][8], axis=1)
    sen_dis_feat["kurtosis_q"] = sen_dis_feat_tmp.apply(
        lambda row: row["sen_dis_tmp"][9], axis=1)
    sen_dis_feat["kurtosis_t"] = sen_dis_feat_tmp.apply(
        lambda row: row["sen_dis_tmp"][10], axis=1)

    return sen_dis_feat


def my_feat_calc_func(RawData):
    
    multiprocessing_nums = 16
    df_parts = np.array_split(RawData, multiprocessing_nums)
    with Pool(processes=multiprocessing_nums) as pool:
        result_parts = pool.map(apply_fun, df_parts)
    pool.join()

    result_parallel = pd.concat(result_parts)

    return result_parallel

In [ ]:
# 以下是生成title count特征

In [ ]:
import pandas as pd
import numpy as np
import gc


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


train_size = 1000000000
test1_size = 20000000
test_final_size = 100000000

# todo 需要修改路径
train_df = pd.read_hdf('/home/kesci/work/featureMap/train/title_code_train_feat.h5', key='data')
test_1_df = pd.read_hdf('/home/kesci/work/featureMap/test1/title_code_test1_feat.h5', key='data')
test_final_df = pd.read_hdf('/home/kesci/work/post_4kw_data/test2/title_code_test2_feat.h5', key='data')

all_data = pd.concat([train_df, test_1_df, test_final_df], ignore_index=True)
del train_df, test_1_df, test_final_df
gc.collect()

all_data = reduce_mem_usage(all_data)

count_feature = pd.DataFrame()

for feature in ['title_code']:
    print('计算' + feature + '点击次数')
    count_feature[feature + '_count'] = all_data.groupby(feature)[feature].transform('count')
    count_feature = reduce_mem_usage(count_feature)

train_count_feature = count_feature[:train_size]
test1_count_feature = count_feature[train_size:train_size + test1_size]
test_final_count_feature = count_feature[train_size + test1_size:]
print(f'train shape = {train_count_feature.shape}')
print(f'test1 shape = {test1_count_feature.shape}')
print(f'test_final shape = {test_final_count_feature.shape}')

train_count_feature.to_hdf('/home/kesci/work/featureMap/train/title_count_all.h5', index=None, key='data', complevel=9)
test1_count_feature.to_hdf('/home/kesci/work/post_4kw_data/test1/test1_title_count_final.h5', index=None, key='data', complevel=9)
test_final_count_feature.to_hdf('/home/kesci/work/post_4kw_data/test2/test2_title_count_final.h5', index=None, key='data', complevel=9)

In [ ]:
# 以下是生成 query count 特征

In [ ]:
import pandas as pd
import numpy as np
import gc


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


train_size = 1000000000
test1_size = 20000000
test_final_size = 100000000

# todo 需要修改路径
train_df = pd.read_hdf('/home/kesci/work/featureMap/train/query_code_train_feat.h5')
test_1_df = pd.read_hdf('/home/kesci/work/featureMap/test1/query_code_test1_feat.h5')
test_final_df = pd.read_hdf('/home/kesci/work/post_4kw_data/test2/query_code_test2_feat.h5')


all_data = pd.concat([train_df, test_1_df, test_final_df], ignore_index=True)
del train_df, test_1_df, test_final_df
gc.collect()

all_data = reduce_mem_usage(all_data)

count_feature = pd.DataFrame()

for feature in ['query_code']:
    print('计算' + feature + '点击次数')
    count_feature[feature + '_count'] = all_data.groupby(feature)[feature].transform('count')
    count_feature = reduce_mem_usage(count_feature)

train_count_feature = count_feature[:train_size]
test1_count_feature = count_feature[train_size:train_size + test1_size]
test_final_count_feature = count_feature[train_size + test1_size:]
print(f'train shape = {train_count_feature.shape}')
print(f'test1 shape = {test1_count_feature.shape}')
print(f'test_final shape = {test_final_count_feature.shape}')

test1_count_feature.to_hdf('/home/kesci/work/post_4kw_data/test1/test1_query_count_sr.h5', index=None, key='data', complevel=9)
train_count_feature.to_hdf('/home/kesci/work/featureMap/train/query_count_all_sr.h5', index=None, key='data', complevel=9)
test_final_count_feature.to_hdf('/home/kesci/work/post_4kw_data/test2/test2_query_count_sr.h5', index=None, key='data', complevel=9)

In [ ]:
# 以下是生成unique特征

In [ ]:
import pandas as pd
import numpy as np
import gc


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
                start_mem - end_mem) / start_mem))
    return df


train_size = 1000000000
test1_size = 20000000
test_final_size = 100000000

# todo 需要修改路径
train_title = pd.read_hdf('/home/kesci/work/featureMap/train/title_code_train_feat.h5', key='data')
test_1_title = pd.read_hdf('/home/kesci/work/featureMap/test1/title_code_test1_feat.h5', key='data')
test_final_title = pd.read_hdf('/home/kesci/work/post_4kw_data/test2/title_code_test2_feat.h5', key='data')

train_query = pd.read_hdf('/home/kesci/work/featureMap/train/query_code_train_feat.h5')
test_1_query = pd.read_hdf('/home/kesci/work/featureMap/test1/query_code_test1_feat.h5')
test_final_query = pd.read_hdf('/home/kesci/work/post_4kw_data/test2/query_code_test2_feat.h5')

train_df = pd.concat([train_query, train_title], axis=1)
test_1_df = pd.concat([test_1_query, test_1_title], axis=1)
test_final_df = pd.concat([test_final_query, test_final_title], axis=1)

del train_query, train_title, test_1_query, test_1_title, test_final_query, test_final_title
gc.collect()
all_data = pd.concat([train_df, test_1_df, test_final_df], ignore_index=True)
del train_df, test_1_df, test_final_df
gc.collect()

all_data = reduce_mem_usage(all_data)

nunique_feature = pd.DataFrame()


# 下面两步 可能需要注释后分两次运行 一次运行可能会炸内存
nunique_feature['query_nunique_title'] = all_data.groupby('query_code').title_code.transform('nunique')
nunique_feature = reduce_mem_usage(nunique_feature)
nunique_feature['title_nunique_query'] = all_data.groupby('title_code').query_code.transform('nunique')
nunique_feature = reduce_mem_usage(nunique_feature)

train_nunique_feature = nunique_feature[:train_size]
test1_nunique_feature = nunique_feature[train_size:train_size + test1_size]
test_final_nunique_feature = nunique_feature[train_size + test1_size:]
print(f'train shape = {train_nunique_feature.shape}')
print(f'test1 shape = {test1_nunique_feature.shape}')
print(f'test_final shape = {test_final_nunique_feature.shape}')

In [ ]:
# 以下是生成test1和train的title_code的转化率  注意一下需要改一下路径 就是title_code的

In [ ]:
import pandas as pd
train_label = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[4], names=['label'])

train_data = pd.read_csv('/home/kesci/work/featureMap/train/title_code_train_feat.csv.bz2')
test_df = pd.read_csv('/home/kesci/work/featureMap/test1/title_code_test1_feat.csv.bz2')

train_df = pd.concat([train_data,train_label],axis=1)
del train_data,train_label
import gc
gc.collect()

def downcast_data(D):
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    return D
train_df = downcast_data(train_df)
test_df = downcast_data(test_df)

import time

import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
import scipy.special as special


def log(log: str):
    print(log)


def time_log(time_elapsed):
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  # 打印出来时间


def log_event(event: str):
    log(event)


class HyperParam(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def sample_from_beta(self, alpha, beta, num, imp_upperbound):
        sample = np.random.beta(alpha, beta, num)
        I = []
        C = []
        for click_ratio in sample:
            imp = np.random.random() * imp_upperbound
            # imp = imp_upperbound
            click = imp * click_ratio
            I.append(imp)
            C.append(click)
        return I, C

    def update_from_data_by_FPI(self, tries, success, iter_num, epsilon):
        '''estimate alpha, beta using fixed point iteration'''
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(tries, success, self.alpha, self.beta)
            if abs(new_alpha - self.alpha) < epsilon and abs(new_beta - self.beta) < epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta

    def __fixed_point_iteration(self, tries, success, alpha, beta):
        '''fixed point iteration'''
        sumfenzialpha = 0.0
        sumfenzibeta = 0.0
        sumfenmu = 0.0
        for i in range(len(tries)):
            sumfenzialpha += (special.digamma(success[i] + alpha) - special.digamma(alpha))
            sumfenzibeta += (special.digamma(tries[i] - success[i] + beta) - special.digamma(beta))
            sumfenmu += (special.digamma(tries[i] + alpha + beta) - special.digamma(alpha + beta))

        return alpha * (sumfenzialpha / sumfenmu), beta * (sumfenzibeta / sumfenmu)

    def update_from_data_by_moment(self, tries, success):  # tries尝试了多少次ctr  success 命中了多少次ctr
        '''estimate alpha, beta using moment estimation'''
        mean, var = self.__compute_moment(tries, success)
        # print 'mean and variance: ', mean, var
        # self.alpha = mean*(mean*(1-mean)/(var+0.000001)-1)
        self.alpha = (mean + 0.000001) * ((mean + 0.000001) * (1.000001 - mean) / (var + 0.000001) - 1)
        # self.beta = (1-mean)*(mean*(1-mean)/(var+0.000001)-1)
        self.beta = (1.000001 - mean) * ((mean + 0.000001) * (1.000001 - mean) / (var + 0.000001) - 1)

    def __compute_moment(self, tries, success):
        '''
        moment estimation
        '''
        ctr_list = []
        var = 0.0
        for i in range(len(tries)):
            ctr_list.append(float(success[i]) / (tries[i] + 0.000000001))
        mean = sum(ctr_list) / len(ctr_list)
        for ctr in ctr_list:
            var += pow(ctr - mean, 2)

        return mean, var / (len(ctr_list) - 1)


def merge_test(train_df, test_df, feature_name, is_fill_na):
    temp = train_df.groupby(feature_name, as_index=False)['label'].agg(
        {feature_name + '_label_count': 'sum', feature_name + '_all_count': 'count'})
    HP = HyperParam(1, 1)
    HP.update_from_data_by_moment(temp[feature_name + '_all_count'].values,
                                  temp[feature_name + '_label_count'].values)  # 矩估计
    temp[feature_name + '_convert'] = (temp[feature_name + '_label_count'] + HP.alpha) / (
            temp[feature_name + '_all_count'] + HP.alpha + HP.beta)
    temp = temp[[feature_name, feature_name + '_convert', feature_name + '_label_count']].drop_duplicates()
    test_df = pd.merge(test_df, temp, on=[feature_name], how='left')
    if is_fill_na:
        test_df[feature_name + '_convert'].fillna(HP.alpha / (HP.alpha + HP.beta), inplace=True)
    return test_df


def merge_train(kfold_4_df, kfold_1_df, feature_name, is_fill_na):  # 用其他训练集的四折的数据去merge一折的
    temp = kfold_4_df.groupby(feature_name, as_index=False)['label'].agg(
        {feature_name + '_label_count': 'sum', feature_name + '_all_count': 'count'})
    HP = HyperParam(1, 1)
    HP.update_from_data_by_moment(temp[feature_name + '_all_count'].values,
                                  temp[feature_name + '_label_count'].values)  # 矩估计
    temp[feature_name + '_convert'] = (temp[feature_name + '_label_count'] + HP.alpha) / (
            temp[feature_name + '_all_count'] + HP.alpha + HP.beta)
    temp = temp[[feature_name, feature_name + '_convert', feature_name + '_label_count']].drop_duplicates()
    kfold_1_df = pd.merge(kfold_1_df, temp, on=[feature_name], how='left')
    if is_fill_na:
        kfold_1_df[feature_name + '_convert'].fillna(HP.alpha / (HP.alpha + HP.beta), inplace=True)
    return kfold_1_df
    
n_splits = 5
is_shuffle = False
feature_name = 'title_code'
is_fill_nan = True

log_event('计算 test ' + feature_name + ' 转换率')
since = time.time()
test_df = merge_test(train_df, test_df, feature_name, is_fill_nan)
time_elapsed = time.time() - since
time_log(time_elapsed)

kf = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=19951024)

if is_shuffle:
    train_df['old_index'] = train_df.index

temp_list = []
for index, (fold_4, fold_1) in enumerate(kf.split(train_df)):
    log_event(f'计算 train fold {(index + 1)} ' + feature_name + ' 转换率')
    since = time.time()
    temp_df = merge_train(train_df.iloc[fold_4], train_df.iloc[fold_1], feature_name, is_fill_nan)
    temp_list.append(temp_df)
    time_elapsed = time.time() - since
    time_log(time_elapsed)

result_df = pd.concat(temp_list, ignore_index=True)

if is_shuffle:
    result_df.sort_values(by='old_index', inplace=True)
    result_df = result_df.reset_index(drop=True)
    
result_df[['title_code_convert','title_code_label_count']].to_hdf('./sunrui/title_convert_train.h5', index=None, key='data',complevel=9)
test_df[['title_code_convert','title_code_label_count']].to_hdf('./sunrui/title_convert_test.h5', index=None, key='data',complevel=9)

In [ ]:
# 一篇文章的质量score计算源码（10折生成前一亿）
# calcScore(dataSet,testSet)
# 读取queryid、title_code、lable三列特征的dataframe 作为dataSet参数
# 读取 title_code 该列特征的dataframe 作为testSet参数
# 这里需要计算四次：
#    1、第二亿到第十亿的queryid、title_code、lable作为dataSet参数，第一亿的title_code作为testSet参数
#    2、第一亿到第九亿的queryid、title_code、lable作为dataSet参数，第十亿的title_code作为testSet参数
#    3、第一亿到第十亿的queryid、title_code、lable作为dataSet参数，test1的title_code作为testSet参数
#    4、第二亿到第十亿的queryid、title_code、lable作为dataSet参数，test2的title_code作为testSet参数

In [ ]:
import pandas as pd
import gc

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def calcScore(dataSet,testSet):
    dataSet['queryid_num']=dataSet.groupby(by='query_id').label.transform('count')
    dataSet['queryid_click_num']=dataSet.groupby(by='query_id').label.transform('sum')
    dataSet=dataSet.drop(columns=["query_id"])
    print("ok1")
    gc.collect()
    dataSet = reduce_mem_usage(dataSet,verbose=True)
    
    dataSet['queryid_click_rate'] = dataSet['queryid_click_num']/dataSet['queryid_num']
    print("ok2")
    dataSet=dataSet.drop(columns=["queryid_num","queryid_click_num"])
    gc.collect()
    dataSet = reduce_mem_usage(dataSet,verbose=True)
    
    dataSet['title_queryid_score']= dataSet['label']- dataSet['queryid_click_rate']
    dataSet=dataSet.drop(columns=["label"])
    gc.collect()
    print("ok3")
    dataSet = reduce_mem_usage(dataSet,verbose=True)
    
    feature='title_code'
    dataSet[feature+'_score'] = dataSet.groupby(by=feature)['title_queryid_score'].transform('sum')
    dataSet=reduce_mem_usage(dataSet,verbose=True)
    dataSet=dataSet[[feature,feature+'_score']].drop_duplicates()
    dataSet = reduce_mem_usage(dataSet,verbose=True)
    gc.collect()
    
    testSet = pd.merge(testSet, dataSet, on=[feature], how='left')
    
    return testSet

In [ ]:
# 一篇文章的点击数、出现数目 计算源码（10折生成前一亿）
# calcScorecount(dataSet,testSet)
# 读取 queryid、title_code、lable三列特征的dataframe 作为dataSet参数
# 读取 title_code 该列特征的dataframe 作为testSet参数
# 这里需要计算四次：
#    1、第二亿到第十亿的queryid、title_code、lable作为dataSet参数，第一亿的title_code作为testSet参数
#    2、第一亿到第九亿的queryid、title_code、lable作为dataSet参数，第十亿的title_code作为testSet参数
#    3、第一亿到第十亿的queryid、title_code、lable作为dataSet参数，test1的title_code作为testSet参数
#    4、第二亿到第十亿的queryid、title_code、lable作为dataSet参数，test2的title_code作为testSet参数

In [ ]:
import pandas as pd
import gc

def reduce_mem_usage(D,verbose=True):
    start_mem = D.memory_usage().sum() / 1024**2
    for c, d in zip(D.columns, D.dtypes):
        if d.kind == 'f':
            D[c] = pd.to_numeric(D[c], downcast='float')
        elif d.kind == 'i':
            D[c] = pd.to_numeric(D[c], downcast='signed')
    end_mem = D.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return D

def calcScorecount(dataSet,testSet):
    dataSet[feature+'_count']=dataSet.groupby(by=feature).label.transform('count')
    print("Ok1")
    dataSet[feature+'_click_num']=dataSet.groupby(by=feature).label.transform('sum')
    print("ok2")
    dataSet=dataSet[[feature,feature+'_count',feature+'_click_num']].drop_duplicates()
    dataSet = reduce_mem_usage(dataSet,verbose=True)
    gc.collect()
    
    testSet = pd.merge(testSet, dataSet, on=[feature], how='left')
    return testSet

In [ ]:
# 以下是生成SIF特征的词频表

In [ ]:
import pandas as pd
import gc
from tqdm import tqdm


words_dict = {}


text_data1 = pd.read_csv('/home/kesci/input/bytedance/train_final.csv', usecols=[1, 3],
                         names=['query', 'title'], nrows=100000000)


query_list = text_data1['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = text_data1['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list

del text_data1
gc.collect()

text_data2 = pd.read_csv('/home/kesci/work/word2vec/post_10kw.csv')


query_list = text_data2['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = text_data2['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del text_data2
gc.collect()

test = pd.read_csv('/home/kesci/input/bytedance/test_final_part1.csv', usecols=[1, 3],
                   names=['query', 'title'])


query_list = test['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = test['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del test
gc.collect()

test2 = pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv', usecols=[1, 3],
                    names=['query', 'title'])


query_list = test2['query'].drop_duplicates().values.tolist()

for item in tqdm(query_list):
    query = item.split()
    for word in query:
        words_dict[word] = words_dict.get(word, 0) + 1

del query_list
gc.collect()

title_list = test2['title'].drop_duplicates().values.tolist()

for item in tqdm(title_list):
    title = item.split()
    for word in title:
        words_dict[word] = words_dict.get(word, 0) + 1

del title_list
del test2
gc.collect()

import operator

sorted_words_dict = sorted(words_dict.items(), key=operator.itemgetter(1), reverse=True)

from tqdm import tqdm

with open('/home/kesci/work/sif_embbeding/word_freq_final.txt', 'w') as f: # trianpre10kw post10kw test1 test2
    for word, freq in tqdm(sorted_words_dict):
        f.write(word + ' ' + str(freq) + '\n')

In [ ]:
# 以下生成SIF句向量的cos_dis l2_dis特征 
# sif参考ppt中的内容

In [ ]:
import numpy as np
from gensim.models import KeyedVectors
from sklearn.decomposition import TruncatedSVD


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (
            start_mem - end_mem) / start_mem))
    return df


class Params(object):
    def __init__(self):
        self.LW = 1e-5
        self.LC = 1e-5
        self.eta = 0.05

    def __str__(self):
        t = "LW", self.LW, ", LC", self.LC, ", eta", self.eta
        t = list(map(str, t))
        return ' '.join(t)


def getWordmap(w2v_file_name):
    w2v_model = KeyedVectors.load_word2vec_format(w2v_file_name, binary=True)
    word2ids = w2v_model.wv.vocab
    words = {}
    for word, vocab in word2ids.items():
        words[word] = vocab.index
    return (words, w2v_model.vectors)


def getWordWeight(weightfile, a=1e-3):
    if a <= 0:  # when the parameter makes no sense, use unweighted
        a = 1.0

    word2weight = {}
    with open(weightfile) as f:
        lines = f.readlines()
    N = 0
    for i in lines:
        i = i.strip()
        if (len(i) > 0):
            i = i.split()
            if (len(i) == 2):
                word2weight[i[0]] = float(i[1])
                N += float(i[1])
            else:
                print(i)
    for key, value in word2weight.items():
        word2weight[key] = a / (a + value / N)
    return word2weight


def getWeight(words, word2weight):
    weight4ind = {}
    for word, ind in words.items():
        if word in word2weight:
            weight4ind[ind] = word2weight[word]
        else:
            weight4ind[ind] = 1.0
    return weight4ind


def prepare_data(list_of_seqs):
    lengths = [len(s) for s in list_of_seqs]
    n_samples = len(list_of_seqs)
    maxlen = np.max(lengths)
    x = np.zeros((n_samples, maxlen)).astype('int32')
    x_mask = np.zeros((n_samples, maxlen)).astype('float32')
    for idx, s in enumerate(list_of_seqs):
        x[idx, :lengths[idx]] = s
        x_mask[idx, :lengths[idx]] = 1.
    x_mask = np.asarray(x_mask, dtype='float32')
    return x, x_mask


def lookupIDX(words, w):
    return words.get(w, len(words) - 1)


def getSeq(p1, words):
    p1 = p1.split()
    X1 = []
    for i in p1:
        X1.append(lookupIDX(words, i))
    return X1


def sentences2idx(sentences, words):
    """
    Given a list of sentences, output array of word indices that can be fed into the algorithms.
    :param sentences: a list of sentences
    :param words: a dictionary, words['str'] is the indices of the word 'str'
    :return: x1, m1. x1[i, :] is the word indices in sentence i, m1[i,:] is the mask for sentence i (0 means no word at the location)
    """
    seq1 = []
    for i in sentences:
        seq1.append(getSeq(i, words))
    x1, m1 = prepare_data(seq1)
    return x1, m1


def seq2weight(seq, mask, weight4ind):
    weight4ind_np = np.zeros((len(weight4ind),))
    for index, weight in weight4ind.items():
        weight4ind_np[index] = weight

    weight = np.zeros(seq.shape).astype('float32')
    for index, row in enumerate(seq):
        weight[index] = weight4ind_np[row]
    weight = weight * mask
    return weight


def get_weighted_average(We, x, w):
    """
    Compute the weighted average vectors
    :param We: We[i,:] is the vector for word i
    :param x: x[i, :] are the indices of the words in sentence i
    :param w: w[i, :] are the weights for the words in sentence i
    :return: emb[i, :] are the weighted average vector for sentence i
    """
    n_samples = x.shape[0]
    emb = np.zeros((n_samples, We.shape[1]))
    for i in range(n_samples):
        emb[i, :] = w[i, :].dot(We[x[i, :], :]) / np.count_nonzero(w[i, :])
    return emb


def compute_pc(X, npc=1):
    """
    Compute the principal components. DO NOT MAKE THE DATA ZERO MEAN!
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: component_[i,:] is the i-th pc
    """
    svd = TruncatedSVD(n_components=npc, n_iter=7, random_state=0)
    svd.fit(X)
    return svd.components_


def remove_pc(X, npc=1):
    """
    Remove the projection on the principal components
    :param X: X[i,:] is a data point
    :param npc: number of principal components to remove
    :return: XX[i, :] is the data point after removing its projection
    """
    pc = compute_pc(X, npc)
    if npc == 1:
        XX = X - X.dot(pc.transpose()) * pc
    else:
        XX = X - X.dot(pc.transpose()).dot(pc)
    return XX


def SIF_embedding(We, x, w, params):
    """
    Compute the scores between pairs of sentences using weighted average + removing the projection on the first principal component
    :param We: We[i,:] is the vector for word i
    :param x: x[i, :] are the indices of the words in the i-th sentence
    :param w: w[i, :] are the weights for the words in the i-th sentence
    :param params.rmpc: if >0, remove the projections of the sentence embeddings to their first principal component
    :return: emb, emb[i, :] is the embedding for sentence i
    """
    emb = get_weighted_average(We, x, w)
    if params.rmpc > 0:
        emb = remove_pc(emb, params.rmpc)
    return emb


wordfile = '/home/kesci/work/word2vec/word2vec_100.bin'  # word vector file, can be downloaded from GloVe website
weightfile = '/home/kesci/work/sif_embbeding/word_freq_final.txt'  # each line is a word and its frequency
weightpara = 1e-3  # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1  # number of principal components to remove in SIF weighting scheme

# load word vectors
(words, We) = getWordmap(wordfile)
print('getWordmap')
# load word weights
word2weight = getWordWeight(weightfile, weightpara)  # word2weight['str'] is the weight for the word 'str'
weight4ind = getWeight(words, word2weight)  # weight4ind[i] is the weight for the i-th word
print('weight4ind')

import pandas as pd
import gc

chunk_size = 1000000

data_train = pd.read_csv('/home/kesci/input/bytedance/bytedance_contest.final_2.csv', usecols=[1, 3],
                    names=['query', 'title'], chunksize=chunk_size)

from tqdm import tqdm

params = Params()
params.rmpc = rmpc

import time

for index, current_data in enumerate(data_train):
    print(index)
    train_cos_sim = []
    train_l2_sim = []

    sentences = current_data['query'].values.tolist() + current_data['title'].values.tolist()  # 前半部分是query 后半部分是title

    since = time.time()

    x, m = sentences2idx(sentences,
                         words)  # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
    time_elapsed = time.time() - since
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    since = time.time()

    w = seq2weight(x, m, weight4ind)  # get word weights
    # set parameters

    time_elapsed = time.time() - since
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  # 打印出来时间
    # get SIF embedding

    since = time.time()

    embedding = SIF_embedding(We, x, w, params)

    time_elapsed = time.time() - since
    print('complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  # 打印出来时间
    all_len = embedding.shape[0]
    query_emb = embedding[:all_len // 2]
    title_emb = embedding[all_len // 2:]

    for i in tqdm(range(current_data.shape[0])):
        v1 = query_emb[i]
        v2 = title_emb[i]
        cosine_distance = np.dot(v1, v2) / (np.linalg.norm(v1) * (np.linalg.norm(v2)))
        l2_distance = np.linalg.norm(v1 - v2)
        train_cos_sim.append(cosine_distance)
        train_l2_sim.append(l2_distance)

    temp_l2 = pd.DataFrame()
    temp_l2['sif_l2'] = train_l2_sim
    temp_l2 = reduce_mem_usage(temp_l2)
    temp_l2.to_hdf(f'/home/kesci/work/sif_embbeding/sif_100_test2/l2/sif_l2_test2_part{index}.h5', index=None,
                   key='data', complevel=9)

    temp_cos = pd.DataFrame()
    temp_cos['sif_cos'] = train_cos_sim
    temp_cos = reduce_mem_usage(temp_cos)
    temp_cos.to_hdf(f'/home/kesci/work/sif_embbeding/sif_100_test2/cos/sif_cos_test2_part{index}.h5', index=None,
                    key='data', complevel=9)

    del sentences, current_data, embedding, query_emb, title_emb, x, m, w, temp_l2, temp_cos
    gc.collect()